# Validación de Calidad de Datos

Este notebook valida la calidad del dataset estandarizado y genera reportes finales.

## Objetivos
1. Generar reporte completo de calidad de datos
2. Validar rangos y valores lógicos
3. Detectar outliers
4. Validar columnas críticas
5. Documentar transformaciones aplicadas
6. Generar métricas finales del dataset

In [1]:
# Importar librerías
import sys
sys.path.append('../src')

import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
from etl.data_loader import load_from_parquet
from etl.validators import (
    generate_quality_report,
    validate_critical_columns,
    detect_outliers,
    validate_ranges
)
from utils.config import STANDARDIZED_PARQUET_FILE, VALID_RANGES

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. Cargar Datos Estandarizados

In [3]:
import os

print(STANDARDIZED_PARQUET_FILE)
print("Existe?", os.path.exists(STANDARDIZED_PARQUET_FILE))

c:\Users\carlo\Documents\KrozFu\repos\fraud-detection-realestate\notebooks\..\data\processed\igac_standardized.parquet
Existe? False


In [2]:
# Cargar dataset estandarizado
df = load_from_parquet(STANDARDIZED_PARQUET_FILE, use_polars=True)
print(f"Dataset cargado: {df.shape[0]:,} filas, {df.shape[1]} columnas")

Cargando datos desde Parquet: c:\Users\carlo\Documents\KrozFu\repos\fraud-detection-realestate\notebooks\..\data\processed\igac_standardized.parquet


FileNotFoundError: The system cannot find the file specified. (os error 2): ...u\repos\fraud-detection-realestate\notebooks\..\data\processed\igac_standardized.parquet (set POLARS_VERBOSE=1 to see full path)

This error occurred with the following context stack:
	[1] 'parquet scan'
	[2] 'sink'


## 2. Reporte de Calidad General

In [ ]:
# Generar reporte completo de calidad
quality_report = generate_quality_report(df)

## 3. Validación de Columnas Críticas

In [ ]:
# Validar que columnas críticas no tengan nulos
is_valid = validate_critical_columns(df)

if is_valid:
    print("\n✅ TODAS LAS COLUMNAS CRÍTICAS SON VÁLIDAS")
else:
    print("\n⚠️ ALGUNAS COLUMNAS CRÍTICAS TIENEN PROBLEMAS")

## 4. Validación de Rangos

In [ ]:
# Validar rangos de años
year_validation = validate_ranges(
    df,
    'YEAR_RADICA',
    min_val=VALID_RANGES['YEAR_RADICA'][0],
    max_val=VALID_RANGES['YEAR_RADICA'][1]
)

print("\nValidación de rangos de años:")
for key, value in year_validation.items():
    print(f"  {key}: {value}")

In [ ]:
# Validar rangos de valores monetarios
valor_validation = validate_ranges(
    df,
    'VALOR_AJUSTADO',
    min_val=VALID_RANGES['VALOR'][0],
    max_val=VALID_RANGES['VALOR'][1]
)

print("\nValidación de rangos de valores:")
for key, value in valor_validation.items():
    print(f"  {key}: {value}")

## 5. Detección de Outliers

In [ ]:
# Detectar outliers en valores ajustados usando método IQR
df_with_outliers = detect_outliers(df, 'VALOR_AJUSTADO', method='iqr', threshold=1.5)

# Contar outliers
outlier_count = df_with_outliers['VALOR_AJUSTADO_OUTLIER'].sum()
outlier_pct = (outlier_count / len(df)) * 100

print(f"\nOutliers detectados: {outlier_count:,} ({outlier_pct:.2f}%)")

In [ ]:
# Visualizar distribución con outliers marcados
plt.figure(figsize=(14, 6))

# Filtrar valores extremos para mejor visualización
valores = df['VALOR_AJUSTADO'].filter(df['VALOR_AJUSTADO'].is_not_null())
q99 = valores.quantile(0.99)
valores_filtered = valores.filter(valores <= q99).to_list()

plt.subplot(1, 2, 1)
plt.hist(valores_filtered, bins=50, edgecolor='black', alpha=0.7)
plt.xlabel('Valor Ajustado (hasta percentil 99)')
plt.ylabel('Frecuencia')
plt.title('Distribución de Valores Ajustados')

plt.subplot(1, 2, 2)
plt.boxplot(valores_filtered)
plt.ylabel('Valor Ajustado')
plt.title('Boxplot de Valores Ajustados')

plt.tight_layout()
plt.show()

## 6. Análisis de Completitud por Columna

In [ ]:
# Calcular porcentaje de completitud por columna
completeness = []
for col in df.columns:
    null_count = df[col].null_count()
    completeness_pct = ((len(df) - null_count) / len(df)) * 100
    completeness.append({
        'Columna': col,
        'Completitud_%': round(completeness_pct, 2)
    })

completeness_df = pl.DataFrame(completeness).sort('Completitud_%')

print("\nCompletitud por columna (ordenado de menor a mayor):")
print(completeness_df)

In [ ]:
# Visualizar completitud
plt.figure(figsize=(12, 8))
plt.barh(completeness_df['Columna'].to_list(), completeness_df['Completitud_%'].to_list())
plt.xlabel('Completitud (%)')
plt.title('Completitud de Datos por Columna')
plt.axvline(x=95, color='r', linestyle='--', label='95% threshold')
plt.legend()
plt.tight_layout()
plt.show()

## 7. Métricas Finales del Dataset

In [ ]:
# Resumen ejecutivo del dataset
print("\n" + "="*60)
print("RESUMEN EJECUTIVO DEL DATASET FINAL")
print("="*60)

print(f"\n📊 Dimensiones:")
print(f"   - Total de registros: {len(df):,}")
print(f"   - Total de columnas: {len(df.columns)}")

print(f"\n📅 Rango Temporal:")
print(f"   - Año mínimo: {df['YEAR_RADICA'].min()}")
print(f"   - Año máximo: {df['YEAR_RADICA'].max()}")

print(f"\n🌍 Cobertura Geográfica:")
print(f"   - Departamentos únicos: {df['DEPARTAMENTO'].n_unique()}")
print(f"   - Municipios únicos: {df['MUNICIPIO'].n_unique()}")

print(f"\n💰 Valores Monetarios (Ajustados):")
print(f"   - Valor mínimo: ${df['VALOR_AJUSTADO'].min():,.0f}")
print(f"   - Valor máximo: ${df['VALOR_AJUSTADO'].max():,.0f}")
print(f"   - Valor promedio: ${df['VALOR_AJUSTADO'].mean():,.0f}")
print(f"   - Valor mediano: ${df['VALOR_AJUSTADO'].median():,.0f}")

print(f"\n✅ Calidad de Datos:")
total_nulls = sum([df[col].null_count() for col in df.columns])
total_cells = len(df) * len(df.columns)
completeness_overall = ((total_cells - total_nulls) / total_cells) * 100
print(f"   - Completitud general: {completeness_overall:.2f}%")
print(f"   - Duplicados: 0 (eliminados en limpieza)")
print(f"   - Outliers detectados: {outlier_count:,} ({outlier_pct:.2f}%)")

print("\n" + "="*60)

## 8. Documentación de Transformaciones

In [ ]:
# Documentar transformaciones aplicadas
transformations = """
TRANSFORMACIONES APLICADAS AL DATASET
======================================

1. LIMPIEZA DE DATOS:
   - Normalización de nombres de municipios y departamentos
   - Parseo de fechas a formato datetime
   - Limpieza de valores numéricos
   - Eliminación de duplicados basado en PK
   - Manejo de valores nulos en columnas críticas

2. ESTANDARIZACIÓN:
   - Ajuste de valores monetarios por IPC (año base: 2024)
   - Normalización de códigos DIVIPOLA a 5 dígitos
   - Creación de campos temporales derivados:
     * MES_RADICA
     * TRIMESTRE_RADICA
     * SEMESTRE_RADICA
     * DIA_SEMANA_RADICA
   - Creación de clave geográfica (GEO_KEY)
   - Cálculo de indicador de alto valor (ALTO_VALOR)

3. VALIDACIÓN:
   - Validación de rangos de años (2015-2025)
   - Validación de valores monetarios
   - Detección de outliers usando método IQR
   - Verificación de completitud de datos
"""

print(transformations)

## Conclusiones

### ✅ Dataset Validado y Listo para Análisis

El dataset ha pasado por un proceso completo de ETL:

1. **Exploración**: Análisis inicial de estructura y características
2. **Limpieza**: Normalización, eliminación de duplicados, manejo de nulos
3. **Estandarización**: Ajuste por inflación, creación de campos derivados
4. **Validación**: Verificación de calidad y detección de anomalías

### 📈 Próximos Pasos

1. **Fase 3**: Definición de la "Normalidad"
   - Análisis estadístico por municipio
   - Cálculo de métricas base (promedio m², desviación estándar)
   - Identificación de patrones estacionales

2. **Fase 4**: Detección de Anomalías
   - Diseño de modelos de detección
   - Implementación de reglas de negocio
   - Validación de resultados

3. **Fase 5**: Visualización y Monitoreo
   - Dashboard interactivo
   - Sistema de alertas
   - Reportes automatizados